# CW + Diversity Regularization on CIFAR10

In [1]:
import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler

import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

import pickle
import datetime
import glob
import os
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('precision', 10)

# custom code imports
from models import *
from neuron_coverage import *

import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

device = torch.device("cpu")
if torch.cuda.is_available():
    print('CUDA is available!')
    device = torch.device("cuda")
else:
    print('CUDA is not available...')

CUDA is available!


# Generate Data

```
conda activate deepxplore

cd C:\Users\Fabrice\Documents\GitHub\deepxplore\Driving

python gen_diff.py light 2 0.5 10 1000 50 0
python gen_diff.py occl 2 0.5 10 1000 50 0
python gen_diff.py blackout 2 0.5 100 1000 50 0

conda activate deepxplore

cd C:\Users\Fabrice\Documents\GitHub\deepxplore\MNIST

python gen_diff.py light 2 0.5 0.5 1000 50 0
python gen_diff.py occl 2 0.5 0.5 1000 50 0
python gen_diff.py blackout 2 0.5 0.5 1000 50 0
```

# Load Data

### MNIST

In [2]:
mnist_dir = r"C:/Users/Fabrice/Documents/GitHub/deepxplore/MNIST/generated_inputs/"

In [3]:
with open(mnist_dir + 'results_MNIST_light.pkl.', 'rb') as handle:
    results_MNIST_light = pickle.load(handle)
    
with open(mnist_dir + 'results_MNIST_occl.pkl.', 'rb') as handle:
    results_MNIST_occl = pickle.load(handle)
    
with open(mnist_dir + 'results_MNIST_blackout.pkl.', 'rb') as handle:
    results_MNIST_blackout = pickle.load(handle)

In [4]:
len(results_MNIST_light['adversaries'])

100

In [5]:
len(results_MNIST_occl['adversaries'])

100

In [6]:
len(results_MNIST_blackout['adversaries'])

100

In [7]:
np.unique(results_MNIST_light['targets'], return_counts=True)

(array([0, 1, 4, 6, 7, 9], dtype=uint8),
 array([10, 20, 19, 18, 20, 13], dtype=int64))

In [8]:
np.unique(results_MNIST_occl['targets'], return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([10, 10, 12, 10,  6, 12, 12,  3, 12, 13], dtype=int64))

In [9]:
np.unique(results_MNIST_blackout['targets'], return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([11,  8,  8, 18,  5,  8,  5,  6, 16, 15], dtype=int64))

### Driving

In [10]:
driving_dir = r"C:/Users/Fabrice/Documents/GitHub/deepxplore/Driving/generated_inputs/"

with open(driving_dir + 'results_Driving_light.pkl.', 'rb') as handle:
    results_Driving_light = pickle.load(handle)
    
with open(driving_dir + 'results_Driving_occl.pkl.', 'rb') as handle:
    results_Driving_occl = pickle.load(handle)
    
with open(driving_dir + 'results_Driving_blackout.pkl.', 'rb') as handle:
    results_Driving_blackout = pickle.load(handle)

In [11]:
len(results_Driving_light['adversaries'])

100

In [12]:
len(results_Driving_occl['adversaries'])

100

In [13]:
len(results_Driving_blackout['adversaries'])

100

In [14]:
np.unique(results_Driving_light['targets'], return_counts=True)

(array([-0.37728189, -0.37366511, -0.37015445, -0.36509831, -0.35592766,
        -0.345652  , -0.34200869, -0.33472228, -0.32987318, -0.32465709,
        -0.32411876, -0.32280696, -0.32002955, -0.31699471, -0.31380253,
        -0.29695919, -0.29579809, -0.28928785, -0.28299015, -0.27832876,
        -0.2727933 , -0.27188941, -0.26611465, -0.26206492, -0.25137091,
        -0.25076337, -0.23023227, -0.22462393, -0.2197968 , -0.21745265,
        -0.20198678, -0.1885332 , -0.17852194, -0.17794751, -0.17002803,
        -0.16833934, -0.16568308, -0.14385353, -0.1164897 , -0.10962802,
        -0.10557798, -0.09934618, -0.0963665 , -0.09135705, -0.08798349,
        -0.08288936, -0.07098416, -0.06692124, -0.05540992, -0.05406632,
        -0.04610964, -0.04592619, -0.03379836, -0.02579822, -0.02280141,
        -0.02140156, -0.01995617, -0.01782793, -0.01404086, -0.00683715,
         0.00502623,  0.00843862,  0.01852022,  0.02438788,  0.02874173,
         0.02892042,  0.03967557,  0.04666991,  0.0

In [15]:
np.unique(results_Driving_occl['targets'], return_counts=True)

(array([-0.39928845, -0.38095004, -0.37047045, -0.36740017, -0.35700043,
        -0.34517553, -0.33989877, -0.33645611, -0.32976463, -0.32465709,
        -0.32411876, -0.32164437, -0.31907906, -0.30743415, -0.30425362,
        -0.30011493, -0.29392682, -0.28050114, -0.27555969, -0.2727933 ,
        -0.25873454, -0.23294082, -0.23109758, -0.23023227, -0.22737362,
        -0.20735071, -0.20017469, -0.18834042, -0.18740071, -0.18585294,
        -0.18392783, -0.17244475, -0.15133414, -0.15128113, -0.15091985,
        -0.15043932, -0.14507876, -0.13264465, -0.12414477, -0.12028408,
        -0.1177834 , -0.1164897 , -0.11582777, -0.11463397, -0.08617129,
        -0.07978241, -0.06684773, -0.06607332, -0.05927238, -0.05572226,
        -0.05524321, -0.05249653, -0.05212902, -0.04453581, -0.02066263,
         0.00375803,  0.02164353,  0.02587947,  0.02654576,  0.02753325,
         0.0282077 ,  0.0430973 ,  0.04733965,  0.06153425,  0.06235548,
         0.06312626,  0.10545148,  0.11003763,  0.1

In [16]:
np.unique(results_Driving_blackout['targets'], return_counts=True)

(array([-0.39928845, -0.39743738, -0.39628813, -0.3950892 , -0.38511309,
        -0.37044841, -0.36698957, -0.3649463 , -0.35696927, -0.35693913,
        -0.35420642, -0.33588668, -0.33577057, -0.30509204, -0.30389648,
        -0.30312354, -0.29834492, -0.29392682, -0.27436178, -0.26594641,
        -0.26174049, -0.25941073, -0.24414348, -0.23794296, -0.22652213,
        -0.2197968 , -0.21282964, -0.18548831, -0.18287656, -0.17321095,
        -0.16795078, -0.1647337 , -0.16031298, -0.15885947, -0.15551566,
        -0.13264465, -0.1232635 , -0.12111675, -0.1180299 , -0.10320319,
        -0.099399  , -0.09319507, -0.09228707, -0.08972303, -0.08884276,
        -0.07978241, -0.0759249 , -0.05818584, -0.05419977, -0.04739555,
        -0.04469278, -0.03379836, -0.02066263, -0.01782793,  0.00374126,
         0.00843862,  0.01366247,  0.02808919,  0.03219862,  0.03747196,
         0.05343545,  0.05595777,  0.06908067,  0.07213698,  0.0796802 ,
         0.08291583,  0.08530263,  0.11366913,  0.1

In [18]:
np.stack(results_Driving_light['adversaries'])

array([[[[  4,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        ...,

        [[ 33,  32,  28],
         [ 28,  27,  23],
         [ 34,  33,  31],
         ...,
         [108, 106, 109],
         [ 11,   9,  10],
         [ 17,  15,  16]],

        [[ 30,  29,  25],
         [ 26,  25,  21],
         [ 22,  21,  19],
         ...,
         [110, 108, 111],
         [ 62,  60,  61],
         [ 20,  18,  19]],

        [[ 35,  34,  30],
         [ 24,  23,  19],
         [ 24,  23,  21],
         ...,
         [107, 105, 108],
        

# Calculate NC, ASR, IS, FID, OD

```
python _DeepXplore_mnist.py
```

### MNIST

In [8]:
data_dir = "C:/Users/Fabrice/Documents/GitHub/diversity_attacks/assets2/deepxplore/"

with open(data_dir + 'dx_results_mnist_Conv2DNet_blackout_2020.01.27.pkl', 'rb') as handle:
    results_MNIST_light = pickle.load(handle)
    
with open(data_dir + 'dx_results_mnist_Conv2DNet_light_2020.01.27.pkl', 'rb') as handle:
    results_MNIST_occl = pickle.load(handle)
    
with open(data_dir + 'dx_results_mnist_Conv2DNet_occl_2020.01.27.pkl', 'rb') as handle:
    results_MNIST_blackout = pickle.load(handle)
    
all_results_MNIST = results_MNIST_light + results_MNIST_occl + results_MNIST_blackout

In [20]:
# with open('assets/dx_results_mnist_Conv2DNet_blackout_2020.01.27.pkl', 'wb') as handle:
#     pickle.dump(results_MNIST_light, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('assets/dx_results_mnist_Conv2DNet_light_2020.01.27.pkl', 'wb') as handle:
#     pickle.dump(results_MNIST_occl, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('assets/dx_results_mnist_Conv2DNet_occl_2020.01.27.pkl', 'wb') as handle:
#     pickle.dump(results_MNIST_blackout, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
target_features = ['model', 
                   'transformation', 
                   'orig_acc', 
                   'pert_acc', 
                   'attack_success_rate',
                   'neuron_coverage_000',
                   'neuron_coverage_020',
                   'neuron_coverage_050',
                   'neuron_coverage_075',
                   'inception_score',
                   'fid_score_64',
                   'fid_score_2048',
                   'output_diversity',
                   'output_diversity_pct']

In [22]:
df = pd.DataFrame.from_dict(all_results_MNIST)
df[target_features] #.to_clipboard(excel=True)

,model,transformation,orig_acc,pert_acc,attack_success_rate,neuron_coverage_000,neuron_coverage_020,neuron_coverage_050,neuron_coverage_075,inception_score,fid_score_64,fid_score_2048,output_diversity,output_diversity_pct
0,Conv2DNet,light,NA,NA,NA,0.9789231779,0.2282994091,0.0634274458,0.0116217991,3.7388828326,NA,NA,NA,NA
1,Conv2DNet,light,0.99,0.9,0.1,0.9831910703,0.2474064347,0.0677609980,0.0098489823,3.6815822460,3.650078523e-05,0.2799079043,2.200937033,0.955854894
2,Conv2DNet,occl,NA,NA,NA,0.9706500328,0.2008535785,0.0575180565,0.0107025607,3.9937360142,NA,NA,NA,NA
3,Conv2DNet,occl,1,0.84,0.16,0.9011818779,0.2728824688,0.1434668418,0.0256730138,3.2126934132,1.091551583e-05,0.4312131671,1.813505292,0.7875954118
4,Conv2DNet,blackout,NA,NA,NA,0.9787918582,0.2381483913,0.0736703874,0.0145764938,4.0328891039,NA,NA,NA,NA
5,Conv2DNet,blackout,0.99,0.99,0.01,0.9834537098,0.2636244255,0.0812212738,0.0144451740,3.7156955772,4.018588102e-05,0.1923280084,2.248642445,0.976573093


### Driving

In [4]:
with open('assets/dx_results_Driving_Dave_orig_blackout_2020.01.27.pkl', 'rb') as handle:
    results_Driving_light = pickle.load(handle)
    
with open('assets/dx_results_Driving_Dave_orig_light_2020.01.27.pkl', 'rb') as handle:
    results_Driving_occl = pickle.load(handle)
    
with open('assets/dx_results_Driving_Dave_orig_occl_2020.01.27.pkl', 'rb') as handle:
    results_Driving_blackout = pickle.load(handle)
    
all_results_Driving = results_Driving_light + results_Driving_occl + results_Driving_blackout

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [35]:
# with open('assets/dx_results_Driving_Dave_orig_blackout_2020.01.27.pkl', 'wb') as handle:
#     pickle.dump(results_Driving_light, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('assets/dx_results_Driving_Dave_orig_light_2020.01.27.pkl', 'wb') as handle:
#     pickle.dump(results_Driving_occl, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('assets/dx_results_Driving_Dave_orig_occl_2020.01.27.pkl', 'wb') as handle:
#     pickle.dump(results_Driving_blackout, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [36]:
target_features = ['model', 
                   'transformation', 
                   'mse',
                   'orig_acc', 
                   'pert_acc', 
                   'attack_success_rate',
                   'neuron_coverage_000',
                   'neuron_coverage_020',
                   'neuron_coverage_050',
                   'neuron_coverage_075',
                   'inception_score',
                   'fid_score_64',
                   'fid_score_2048',
                   'output_diversity',
                   'output_diversity_pct']

In [38]:
df = pd.DataFrame.from_dict(all_results_Driving)
df[target_features].to_clipboard(excel=True)